In [1217]:
import re, subprocess, pyodbc 
import numpy as np # linear algebra 
from sqlalchemy import func, create_engine
from config import mongoConn, connstr  
import pandas as pd  

In [1218]:
engine = create_engine(connstr) 
def get_df(query):
    conn = engine.connect() 
    df = pd.read_sql(query,con=conn) 
    conn.close()
    return df

In [1219]:
def id_split(s):
    s = s + '000'
    s = re.sub('[^\d]', '', s)  
    return s[:4] 
#print( id_split('5.1-4') )

In [1220]:
def id_num(s):
    if s == '':
        return 0
    s = s.strip().lower().replace(' ', '')
    l = ['a','b','c','d','e','f','g','h']
    d = [.100, 1, 100, 10000]
    v = [0,0,0,0]
    for i, c in enumerate(l):
        s = s.replace(c,f'{i}') 
    l = s.split('.')
    for i, c in enumerate(l):
        v[i] =  int(c) / d[i] 
    return sum(v)  
print(id_num('1.1.1')) 
print(id_num('1.1.10.1')) 

11.01
11.1001


In [1221]:
def normalizedf(df):
    df.drop( columns=[c for c in df.columns if '__' in c ] , inplace=True )   
    cols = list(df)   
    df[cols] = df[cols].astype(str)
    df['IDNUM'] = df['IDTEXT'].apply(id_num)
    df.sort_values('IDNUM',inplace=True)   
    df['QPK'] = range(28200, len(df)+28200 )  
    df['sortpos'] = range(1, len(df)+1)  
    return df

In [1222]:
query = f""" \
    SELECT TOP 5000 
      IDTEXT AS IDTEXT 
    , PK_Question
    , PK_Question as [2021PK_Question]
    , QGROUP
    , '@FormName' AS  FormName
    , '@PK_QGroup' AS  FK_QuestionGroup
    , sortpos AS  sortpos 
    , NULL AS  FK_Question_Picklist
    , CONVERT(NVARCHAR(5), FK_PickListType) AS  FK_PickList 
    , ISNULL(FK_QuestionType, '') AS  FK_QuestionType
    , NULL AS  FK_InputType
    , CONVERT(NVARCHAR(5), FK_PickList) AS FK_PickListType
    , NULL AS  WarningQuestion 
    , NULL AS  ExternalLinkType
    , NULL AS  PK_ExternalLink
    , ISNULL(QText, '') AS  QuestionText
    , ISNULL(help_text, '') AS  help_text 
    FROM vwQuestions WHERE PK_FORM LIKE '2021-A-HVA'  
"""  
dfq  = get_df(query) 
dfq = normalizedf(dfq)
dfq.sort_values(['QGROUP','IDNUM'], inplace=True)
dfq.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfq.csv', index=False)  
dfq[:2] 

,IDTEXT,PK_Question,2021PK_Question,QGROUP,FormName,FK_QuestionGroup,sortpos,FK_Question_Picklist,FK_PickList,FK_QuestionType,FK_InputType,FK_PickListType,WarningQuestion,ExternalLinkType,PK_ExternalLink,QuestionText,help_text,IDNUM,QPK
0,0.1,22575,22575,2431,@FormName,@PK_QGroup,14,None,None,18,None,None,None,None,None,HVA Information,,1.0,28213
2,1,22577,22577,2431,@FormName,@PK_QGroup,15,None,None,9,None,None,None,None,None,What is the purpose of the HVA?,,10.0,28214


In [1223]:
dfh = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='1') 
dfh = normalizedf(dfh) 
dfh['QuestionText'] = ''
dfh[0:2] 

,IDTEXT,FK_QuestionType,FK_PickListType,IDNUM,QPK,sortpos,QuestionText
0,1,9,nan,10.0,28200,1,
1,1.1,17,nan,11.0,28201,2,


In [1224]:
df1 = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='ALL') 
df1 = normalizedf(df1) 
df1[1:2]  # .loc[ df1['IDTEXT'].str.contains('5.') ]

,IDTEXT,QuestionText,QGROUP,IDNUM,QPK,sortpos
1,1,What is the purpose of the HVA?,2431.0,10.0,28201,2


In [1225]:
ppl = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='PurposePickList' )
l=[]
c=0
for i,r in ppl.iterrows():
    if i % 2==1: 
        continue
    c=c+1
    IDTEXT = f"1.1.{ c }"
    QuestionText = f"{ r['DisplayValue'] }"
    l.append( {'IDTEXT':f'{IDTEXT}', 'QuestionText': f"If {QuestionText}, describe:"})
ppl = pd.DataFrame(l)  
ppl.to_csv(r'C:\Users\Tim\Documents\1802\HVA\ppl.csv', index=False)  
ppl[0:2]


,IDTEXT,QuestionText
0,1.1.1,"If .GOV Domain Registration Systems, describe:"
1,1.1.2,"If Agency support tool , describe:"


In [1226]:
dfp = pd.read_csv(r'C:\Users\Tim\Documents\1802\HVA\ppl.csv' )
dfp['IDTEXT'] = dfp['IDTEXT'].apply(lambda s : s.replace('.a','.1'))
dfp['IDTEXT'] = dfp['IDTEXT'].apply(lambda s : s.replace('.b','.2'))
dfp = normalizedf(dfp)  
dfp[:2]

,IDTEXT,QuestionText,IDNUM,QPK,sortpos
0,1.1.1,"If .GOV Domain Registration Systems, describe:",11.01,28200,1
1,1.1.2,"If Agency support tool , describe:",11.02,28201,2


In [1227]:
dfm = pd.merge(df1, dfp, how='outer', on=['IDTEXT', 'QuestionText'], suffixes=(None, '__')) 
dfm = normalizedf(dfm)
dfm.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfm.csv', index=False)  
dfm[0:2]  

,IDTEXT,QuestionText,QGROUP,IDNUM,QPK,sortpos
0,0.1,HVA Information,nan,1.0,28200,1
1,1,What is the purpose of the HVA?,2431.0,10.0,28201,2


In [1228]:
dfa = dfm
dfa = normalizedf(dfa) 
for i,r in dfa.iterrows():
    v=dfa.loc[i, 'QuestionText']
    if v=='':
        dfa.loc[i, 'QuestionText']=dfa.loc[i, 'QuestionText_']
dfa.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfa.csv', index=False)   
dfa['QuestionText'].fillna('')
dfa[0:2] 

,IDTEXT,QuestionText,QGROUP,IDNUM,QPK,sortpos
0,0.1,HVA Information,nan,1.0,28200,1
1,1,What is the purpose of the HVA?,2431.0,10.0,28201,2


In [1229]:
dff = pd.merge(dfa, dfq, how='left', on='IDTEXT', suffixes=('', '_y')) 
dff['FK_QuestionType'].fillna('9', inplace=True)
dff['FK_PickList'].fillna('NULL', inplace=True)
dff['FK_PickListType'].fillna('NULL', inplace=True)
dff['WarningQuestion'].fillna('NULL', inplace=True)
dff['PK_ExternalLink'].fillna('NULL', inplace=True)
dff['PK_Question']=dff['QPK']
dff['FK_InputType'].fillna('NULL', inplace=True) 
dff['help_text'].fillna('NULL', inplace=True) 
dff = dff.fillna('NULL')
dff  = dff.replace('None','NULL')
dff.to_csv(r'C:\Users\Tim\Documents\1802\HVA\dfa.csv', index=False)   
dff[81:82]

,IDTEXT,QuestionText,QGROUP,IDNUM,QPK,sortpos,PK_Question,2021PK_Question,QGROUP_y,FormName,...,FK_QuestionType,FK_InputType,FK_PickListType,WarningQuestion,ExternalLinkType,PK_ExternalLink,QuestionText_y,help_text,IDNUM_y,QPK_y
81,11,Is the HVA system is hosted outside of your Ag...,2431.0,110.0,28281,82,28281,22592,2431,@FormName,...,17,NULL,157,NULL,NULL,NULL,Who is responsible for operating and maintaini...,,110.0,28254.0


In [1230]:
query = f"""  SELECT TOP 1 
    PK_Question, FormName, FK_QuestionGroup,sortpos,identifier_text, FK_QuestionType
    , FK_InputType, FK_PickListType, WarningQuestion, ExternalLinkType, QuestionText, help_text 
    FROM  fsma_Questions 
    """  
df_qmeta = get_df(query) 
cols = ','.join(df_qmeta.columns)
INS = f'INSERT INTO fsma_Questions ({cols}) VALUES '
print(INS)

INSERT INTO fsma_Questions (PK_Question,FormName,FK_QuestionGroup,sortpos,identifier_text,FK_QuestionType,FK_InputType,FK_PickListType,WarningQuestion,ExternalLinkType,QuestionText,help_text) VALUES 


In [1231]:
sql = "" 
for i, r in dff[:84].iterrows():
    s = "(@QID, @FormName, @PK_QGroup, @SORD, N'@IDTEXT', @QTYPE, 1, @PKTYPE, 0, NULL, N'@QTEXT','@HTEXT'),"
    s = s.replace('@QID',  str(r['QPK']))
    s = s.replace('@SORD', str(i+1)) 
    s = s.replace('@QTYPE', r['FK_QuestionType']) 
    s = s.replace('@PKTYPE', r['FK_PickListType']) 
    s = s.replace('@IDTEXT', r['IDTEXT']) 
    s = s.replace('@QTEXT', r['QuestionText'].replace('\'', '`'))  
    s = s.replace('@HTEXT', r['help_text'])  
    sql = sql + s + '\r'
sql = sql.replace("'NULL')", "NULL)")    
sql = sql.replace("N'NULL'", "N''")   
sql = sql.replace("None,", "NULL,")  
sql = f'{INS}\n {sql}\n' 
with open(fr'C:\Users\Tim\Documents\1802\HVA\update.sql', 'w') as f:
    f.write(sql)  



df = pd.read_excel(r'C:\Users\Tim\Documents\1802\HVA\HVA1.xlsx', sheet_name='PurposePickList').fillna('NULL') 
cols = list(df) # Creates list of all column headers
df[cols] = df[cols].astype(str)
df['DisplayCode'] =  df['DisplayValue'].apply(lambda s: re.sub('[^A-Z0-9]','',s.upper()))
df['DisplayCode'] =  df['DisplayCode'].apply(lambda s: re.sub('[AEIOURST]','',s)[:8])
s =''
for i,r, in df.iterrows():
    PK_PickList=r['PK_PickList']
    DisplayCode=r['DisplayCode']
    DisplayValue=r['DisplayValue'] 
    if DisplayValue=='Other':
        DisplayCode='OTH'
    s = s + f"({PK_PickList},@PKT, N'{DisplayCode}',N'{DisplayValue}',{i}, GETDATE(), 1),\n"

print(s)
